In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using GLMakie
using GeometryBasics
using PyCall
using Distances

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMolNotebooks
  ✓ MorphoMolNotebooks
  1 dependency successfully precompiled in 6 seconds. 283 already precompiled.


In [2]:
function get_multichromatic_tetrahedra_v(points, molecule_partitioning)
    py"""
    import numpy as np
    import diode

    def get_multichromatic_tetrahedra(points, molecule_partitioning):        
        def is_multi(sigma):
            print(len(set(next(filter(lambda a : a >= v, molecule_partitioning)) for v in sigma)))
            return len(set(next(filter(lambda a : a >= v, molecule_partitioning)) for v in sigma)) >= 2
            #return len(set(v // n_atoms_per_mol for v in sigma)) >= 2
        points = np.asarray(points)
        tetrahedra = [vs for (vs,fs) in diode.fill_alpha_shapes(points) if len(vs) == 4 and is_multi(vs)]
        return tetrahedra
    """
    tets = py"get_multichromatic_tetrahedra"(points, molecule_partitioning)
    tets = [t .+ 1 for t in tets]
end

get_multichromatic_tetrahedra_v (generic function with 1 method)

In [204]:
function get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    py"""
    import numpy as np
    import diode

    def get_multichromatic_tetrahedra(points, n_atoms_per_mol):        
        def is_multi(sigma):
            return len(set(v // n_atoms_per_mol for v in sigma)) >= 2
        points = np.asarray(points)
        tetrahedra = [vs for (vs,fs) in diode.fill_alpha_shapes(points) if len(vs) == 4 and is_multi(vs)]
        return tetrahedra
    """
    tets = py"get_multichromatic_tetrahedra"(points, n_atoms_per_mol)
    tets = [t .+ 1 for t in tets]
end

function get_chromatic_partitioning(tet, n_atoms_per_mol)
    divs = [div(v - 1, n_atoms_per_mol) for v in tet]
    parts = Dict(k => Vector{Int}([]) for k in unique(divs))
    for (i, d) in enumerate(divs)
        push!(parts[d], tet[i])
    end
    sort([v for v in values(parts)], by=length, rev = true)
end

function get_barycenter(points, vertices) 
    Point3f(sum(points[vertices]) / length(vertices))
end

function get_or_create_bc_simplex_id_and_val!(partitioning::Vector{Vector{Int}}, uob_to_barycenter_simplices, points)
    simplex = Tuple(sort(vcat(partitioning...)))
    try
        (id, val) = uob_to_barycenter_simplices[simplex]
        return true, (id, val)
    catch
        id = length(uob_to_barycenter_simplices) + 1
        if length(partitioning) == 2
            part_one, part_two = partitioning
            val = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_two)) / 2.0
        elseif length(partitioning) == 3
            part_one, part_two, part_three = partitioning
            a = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_two))
            b = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_three))
            c = euclidean(get_barycenter(points, part_two), get_barycenter(points, part_three))
            val = (a + b + c) / 3.0
        elseif length(partitioning) == 4
            part_one, part_two, part_three, part_four = partitioning
            a = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_two))
            b = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_three))
            c = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_four))
            d = euclidean(get_barycenter(points, part_two), get_barycenter(points, part_three))
            e = euclidean(get_barycenter(points, part_two), get_barycenter(points, part_four))
            f = euclidean(get_barycenter(points, part_three), get_barycenter(points, part_four))
            val = (a + b + c + d + e + f) / 6.0
        end
        uob_to_barycenter_simplices[simplex] = (id, val)
        return false, (id, val)
    end
end 

function get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol::Int)
    barycenters = Vector{Point3f}([])
    filtration = Set{Tuple{Vector{Int32}, Float64}}([])
    uob_to_barycenter_simplices = Dict{Any, Any}()
    for vs in eachrow(mc_tets)
        parts = get_chromatic_partitioning(vs, n_atoms_per_mol)
        if length(parts) == 2
            part_one, part_two = parts
            if length(part_one) == length(part_two) == 2
                #Get Simplices and Values for even split
                u, v = part_one
                x, y = part_two
    
                created = Vector{Bool}([])
                vertices = Vector{Tuple{Int, Float64}}([])
                for uob_simplex in [[[u,v],[x,y]], [[x],[v]], [[x,y],[v]], [[y],[v]], [[y],[u,v]], [[y],[u]], [[x,y],[u]], [[x],[u]], [[x],[u,v]]]
                    exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                    push!(vertices, (id, val))
                    push!(created, !exists)
                end
    
                #Placing the points for visualization
                bc_e = [
                    get_barycenter(points, [x,v]),
                    get_barycenter(points, [y,v]),
                    get_barycenter(points, [y,u]),
                    get_barycenter(points, [x,u])
                ]
    
                barycenters = [barycenters; [
                    get_barycenter(bc_e, [1,2,3,4]),    #u v x y
                    bc_e[1],                            #x v
                    get_barycenter(bc_e, [1,2]),        #x y v
                    bc_e[2],                            #y v
                    get_barycenter(bc_e, [2,3]),        #y u v
                    bc_e[3],                            #y u
                    get_barycenter(bc_e, [3,4]),        #x y u
                    bc_e[4],                            #x u
                    get_barycenter(bc_e, [1,4]),        #x u v
                ][created]]
    
                # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
                edges = [
                    (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                    (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                    (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                    (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                    (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                    (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                    (sort!([vertices[1][1], vertices[8][1]]), minimum([vertices[1][2], vertices[8][2]])),
                    (sort!([vertices[1][1], vertices[9][1]]), minimum([vertices[1][2], vertices[9][2]])),
                    (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                    (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                    (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                    (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                    (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                    (sort!([vertices[7][1], vertices[8][1]]), minimum([vertices[7][2], vertices[8][2]])),
                    (sort!([vertices[8][1], vertices[9][1]]), minimum([vertices[8][2], vertices[9][2]])),
                    (sort!([vertices[9][1], vertices[2][1]]), minimum([vertices[9][2], vertices[2][2]]))
                ]
    
                triangles = [
                    ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                    ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                    ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                    ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                    ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                    ([vertices[1][1], vertices[7][1], vertices[8][1]], minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                    ([vertices[1][1], vertices[8][1], vertices[9][1]], minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                    ([vertices[1][1], vertices[9][1], vertices[2][1]], minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
                ]
                union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
                union!(filtration, Set(edges))
                union!(filtration, Set(triangles))
            elseif 3 == length(part_one) != length(part_two) == 1
                u,v,w = part_one[1], part_one[2], part_one[3]
                x = part_two[1]
    
                created = Vector{Bool}([])
                vertices = Vector{Tuple{Int, Float64}}([])
                for uob_simplex in [[[u,v,w],[x]], [[x],[v]], [[x],[w,v]], [[w],[x]], [[w,u],[x]], [[u],[x]], [[u,v],[x]]]
                    exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                    push!(vertices, (id, val))
                    push!(created, !exists)
                end
    
                bc_e = [
                    get_barycenter(points, [x,v]),      
                    get_barycenter(points, [x,w]),      
                    get_barycenter(points, [u,x])       
                ]
    
                barycenters = [barycenters; [
                    get_barycenter(bc_e, [1,2,3]),  #u v w x
                    bc_e[1],                        #v x
                    get_barycenter(bc_e, [1,2]),    #v w x
                    bc_e[2],                        #w x
                    get_barycenter(bc_e, [2,3]),    #w u x
                    bc_e[3],                        #u x
                    get_barycenter(bc_e, [1,3]),    #u v x
                ][created]]
                
                # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
                edges = [
                    (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                    (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                    (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                    (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                    (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                    (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                    (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                    (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                    (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                    (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                    (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                    (sort!([vertices[7][1], vertices[2][1]]), minimum([vertices[7][2], vertices[2][2]]))
                ]
    
                triangles = [
                    ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                    ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                    ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                    ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                    ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                    ([vertices[1][1], vertices[7][1], vertices[2][1]], minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
                ]
    
                union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
                union!(filtration, Set(edges))
                union!(filtration, Set(triangles))
            else
                error("Invalid partitioning: $vs -> $parts")
            end
        elseif length(parts) == 3
            part_one, part_two, part_three = parts
            a, b = part_one
            u = part_two[1]
            x = part_three[1]

            created = Vector{Bool}([])
            vertices = Vector{Tuple{Int, Float64}}([])
            for uob_simplex in [[[a,b],[u],[x]], [[a],[u]], [[a],[u],[x]], [[a],[x]], [[a,b],[x]], [[b],[x]], [[b],[u],[x]], [[b],[u]], [[a,b],[u]], [[u],[x]]]
                exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                push!(vertices, (id, val))
                push!(created, !exists)
            end

            bc_e = [
                get_barycenter(points, [a,u]), #1
                get_barycenter(points, [a,x]), #2   
                get_barycenter(points, [b,x]), #3
                get_barycenter(points, [b,u]), #4
                get_barycenter(points, [u,x])  #5
            ]

            barycenters = [barycenters; [
                get_barycenter(bc_e, [1,2,3,4,5]),  #a b u x
                bc_e[1],                            #a u 
                get_barycenter(bc_e, [1,2,5]),      #a u x
                bc_e[2],                            #a x
                get_barycenter(bc_e, [2,3]),        #a b x
                bc_e[3],                            #b x
                get_barycenter(bc_e, [3,4,5]),      #b u x
                bc_e[4],                            #b u
                get_barycenter(bc_e, [1,4]),        #a b u
                bc_e[5],                            #u x
            ][created]]

            # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
            edges = [
                (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                (sort!([vertices[1][1], vertices[8][1]]), minimum([vertices[1][2], vertices[8][2]])),
                (sort!([vertices[1][1], vertices[9][1]]), minimum([vertices[1][2], vertices[9][2]])),
                (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                (sort!([vertices[7][1], vertices[8][1]]), minimum([vertices[7][2], vertices[8][2]])),
                (sort!([vertices[8][1], vertices[9][1]]), minimum([vertices[8][2], vertices[9][2]])),
                (sort!([vertices[9][1], vertices[2][1]]), minimum([vertices[9][2], vertices[2][2]])),
                (sort!([vertices[1][1], vertices[10][1]]), minimum([vertices[1][2], vertices[10][2]])),
                (sort!([vertices[3][1], vertices[10][1]]), minimum([vertices[3][2], vertices[10][2]])),
                (sort!([vertices[7][1], vertices[10][1]]), minimum([vertices[7][2], vertices[10][2]])),
            ]

            triangles = [
                ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([vertices[1][1], vertices[7][1], vertices[8][1]], minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                ([vertices[1][1], vertices[8][1], vertices[9][1]], minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                ([vertices[1][1], vertices[9][1], vertices[2][1]], minimum([vertices[1][2], vertices[9][2], vertices[2][2]])),
                ([vertices[1][1], vertices[3][1], vertices[10][1]], minimum([vertices[1][2], vertices[3][2], vertices[10][2]])),
                ([vertices[1][1], vertices[7][1], vertices[10][1]], minimum([vertices[1][2], vertices[7][2], vertices[10][2]])),
            ]
            union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
            union!(filtration, Set(edges))
            union!(filtration, Set(triangles))
        elseif length(parts) == 4
            part_one, part_two, part_three, part_four = parts
            a = part_one[1]
            i = part_two[1]
            u = part_three[1]
            x = part_four[1]

            created = Vector{Bool}([])
            vertices = Vector{Tuple{Int, Float64}}([])
            for uob_simplex in [[[a],[i]], [[a],[u]], [[a],[x]], [[i],[u]], [[i],[x]], [[u],[x]], [[a],[i],[u]], [[a],[i],[x]], [[i],[u],[x]],  [[a],[u],[x]], [[a],[i],[u],[x]]]
                exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                push!(vertices, (id, val))
                push!(created, !exists)
            end

            bc_e = [
                get_barycenter(points, [a,i]), #1
                get_barycenter(points, [a,u]), #2   
                get_barycenter(points, [a,x]), #3
                get_barycenter(points, [i,u]), #4
                get_barycenter(points, [i,x]), #5
                get_barycenter(points, [u,x]), #6
            ]

            barycenters = [barycenters; [
                bc_e[1],                                #a i            1
                bc_e[2],                                #a u            2
                bc_e[3],                                #a x            3
                bc_e[4],                                #i u            4
                bc_e[5],                                #i x            5
                bc_e[6],                                #u x            6
                get_barycenter(bc_e, [1,2,4]),          #a i u          7
                get_barycenter(bc_e, [1,3,5]),          #a i x          8
                get_barycenter(bc_e, [4,5,6]),          #i u x          9
                get_barycenter(bc_e, [2,3,6]),          #a u x          10  
                get_barycenter(bc_e, [1,2,3,4,5,6]),    #a i u x        11
            ][created]]

            # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
            edges = Vector{Tuple{Vector{Int}, Float64}}([])
            for (i, j) in [(1,7), (1,8), (1,11), (2,7), (2,10), (2,11), (3,8), (3,10), (3,11), (4,7), (4,9), (4,11), (5,8), (5,9), (5,11), (6,9), (6,10), (6,11), (7,11), (8,11), (9,11), (10,11)]
                push!(edges, (sort!([vertices[i][1], vertices[j][1]]), minimum([vertices[i][2], vertices[j][2]])))
            end

            triangles = Vector{Tuple{Vector{Int}, Float64}}([])
            for (i, j, k) in [(1,7,11), (1,8,11), (2,7,11), (2,10,11), (3,8,11), (3,10,11), (4,7,11), (4,9,11), (5,8,11), (5,9,11), (6,9,11), (6,10,11)]
                push!(triangles, ([vertices[i][1], vertices[j][1], vertices[k][1]], minimum([vertices[i][2], vertices[j][2], vertices[k][2]])))
            end

            union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
            union!(filtration, Set(edges))
            union!(filtration, Set(triangles))
        end
    end
    barycenters, sort!(sort!(collect(filtration), by = x -> x[1]), by = x -> length(x[1]))
end


function get_barycentric_subdivision_and_filtration_hole(points, mc_tets, n_atoms_per_mol::Int)
    barycenters = Vector{Point3f}([])
    filtration = Set{Tuple{Vector{Int32}, Float64}}([])
    uob_to_barycenter_simplices = Dict{Any, Any}()
    for vs in eachrow(mc_tets)
        parts = get_chromatic_partitioning(vs, n_atoms_per_mol)
        if length(parts) == 2
            part_one, part_two = parts
            if length(part_one) == length(part_two) == 2
                #Get Simplices and Values for even split
                u, v = part_one
                x, y = part_two
    
                created = Vector{Bool}([])
                vertices = Vector{Tuple{Int, Float64}}([])
                for uob_simplex in [[[u,v],[x,y]], [[x],[v]], [[x,y],[v]], [[y],[v]], [[y],[u,v]], [[y],[u]], [[x,y],[u]], [[x],[u]], [[x],[u,v]]]
                    exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                    push!(vertices, (id, val))
                    push!(created, !exists)
                end
    
                #Placing the points for visualization
                bc_e = [
                    get_barycenter(points, [x,v]),
                    get_barycenter(points, [y,v]),
                    get_barycenter(points, [y,u]),
                    get_barycenter(points, [x,u])
                ]
    
                barycenters = [barycenters; [
                    get_barycenter(bc_e, [1,2,3,4]),    #u v x y
                    bc_e[1],                            #x v
                    get_barycenter(bc_e, [1,2]),        #x y v
                    bc_e[2],                            #y v
                    get_barycenter(bc_e, [2,3]),        #y u v
                    bc_e[3],                            #y u
                    get_barycenter(bc_e, [3,4]),        #x y u
                    bc_e[4],                            #x u
                    get_barycenter(bc_e, [1,4]),        #x u v
                ][created]]
    
                # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
                edges = [
                    (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                    (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                    (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                    (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                    (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                    (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                    (sort!([vertices[1][1], vertices[8][1]]), minimum([vertices[1][2], vertices[8][2]])),
                    (sort!([vertices[1][1], vertices[9][1]]), minimum([vertices[1][2], vertices[9][2]])),
                    (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                    (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                    (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                    (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                    (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                    (sort!([vertices[7][1], vertices[8][1]]), minimum([vertices[7][2], vertices[8][2]])),
                    (sort!([vertices[8][1], vertices[9][1]]), minimum([vertices[8][2], vertices[9][2]])),
                    (sort!([vertices[9][1], vertices[2][1]]), minimum([vertices[9][2], vertices[2][2]]))
                ]
    
                triangles = [
                    ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                    ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                    ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                    ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                    ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                    ([vertices[1][1], vertices[7][1], vertices[8][1]], minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                    ([vertices[1][1], vertices[8][1], vertices[9][1]], minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                    ([vertices[1][1], vertices[9][1], vertices[2][1]], minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
                ]
                union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
                union!(filtration, Set(edges))
                union!(filtration, Set(triangles))
            elseif 3 == length(part_one) != length(part_two) == 1
                u,v,w = part_one[1], part_one[2], part_one[3]
                x = part_two[1]
    
                created = Vector{Bool}([])
                vertices = Vector{Tuple{Int, Float64}}([])
                for uob_simplex in [[[u,v,w],[x]], [[x],[v]], [[x],[w,v]], [[w],[x]], [[w,u],[x]], [[u],[x]], [[u,v],[x]]]
                    exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                    push!(vertices, (id, val))
                    push!(created, !exists)
                end
    
                bc_e = [
                    get_barycenter(points, [x,v]),      
                    get_barycenter(points, [x,w]),      
                    get_barycenter(points, [u,x])       
                ]
    
                barycenters = [barycenters; [
                    get_barycenter(bc_e, [1,2,3]),  #u v w x
                    bc_e[1],                        #v x
                    get_barycenter(bc_e, [1,2]),    #v w x
                    bc_e[2],                        #w x
                    get_barycenter(bc_e, [2,3]),    #w u x
                    bc_e[3],                        #u x
                    get_barycenter(bc_e, [1,3]),    #u v x
                ][created]]
                
                # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
                edges = [
                    (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                    (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                    (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                    (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                    (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                    (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                    (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                    (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                    (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                    (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                    (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                    (sort!([vertices[7][1], vertices[2][1]]), minimum([vertices[7][2], vertices[2][2]]))
                ]
    
                triangles = [
                    ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                    ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                    ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                    ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                    ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                    ([vertices[1][1], vertices[7][1], vertices[2][1]], minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
                ]
    
                union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
                union!(filtration, Set(edges))
                union!(filtration, Set(triangles))
            else
                error("Invalid partitioning: $vs -> $parts")
            end
        elseif length(parts) == 3
            part_one, part_two, part_three = parts
            a, b = part_one
            u = part_two[1]
            x = part_three[1]

            created = Vector{Bool}([])
            vertices = Vector{Tuple{Int, Float64}}([])
            for uob_simplex in [[[u],[x]], [[a],[u]], [[a],[u,x]], [[a],[x]], [[a,b],[x]], [[b],[x]], [[b],[u, x]], [[b],[u]], [[a,b],[u]]]
                exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points)
                push!(vertices, (id, val))
                push!(created, !exists)
            end

            bc_e = [
                get_barycenter(points, [u,x]),  #1
                get_barycenter(points, [a,u]), #2
                get_barycenter(points, [a,x]), #3   
                get_barycenter(points, [b,x]), #4
                get_barycenter(points, [b,u]), #5
            ]

            barycenters = [barycenters; [
                bc_e[1],                            #u x
                bc_e[2],                            #a u 
                get_barycenter(bc_e, [2,3]),        #a u x
                bc_e[3],                            #a x
                get_barycenter(bc_e, [3,4]),        #a b x
                bc_e[4],                            #b x
                get_barycenter(bc_e, [4,5]),        #b u x
                bc_e[5],                            #b u
                get_barycenter(bc_e, [2,5]),        #a b u
            ][created]]

            # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
            edges = [
                (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                (sort!([vertices[1][1], vertices[8][1]]), minimum([vertices[1][2], vertices[8][2]])),
                (sort!([vertices[1][1], vertices[9][1]]), minimum([vertices[1][2], vertices[9][2]])),
                (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                (sort!([vertices[7][1], vertices[8][1]]), minimum([vertices[7][2], vertices[8][2]])),
                (sort!([vertices[8][1], vertices[9][1]]), minimum([vertices[8][2], vertices[9][2]])),
                (sort!([vertices[9][1], vertices[2][1]]), minimum([vertices[9][2], vertices[2][2]])),
            ]

            triangles = [
                ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([vertices[1][1], vertices[7][1], vertices[8][1]], minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                ([vertices[1][1], vertices[8][1], vertices[9][1]], minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                ([vertices[1][1], vertices[9][1], vertices[2][1]], minimum([vertices[1][2], vertices[9][2], vertices[2][2]])),
            ]
            union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
            union!(filtration, Set(edges))
            union!(filtration, Set(triangles))
        end
    end
    barycenters, sort!(sort!(collect(filtration), by = x -> x[1]), by = x -> length(x[1]))
end

get_barycentric_subdivision_and_filtration_hole (generic function with 1 method)

In [252]:
function get_interface_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int, show_wireframe = false)
    mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    bcs, filtration = MorphoMol.Energies.get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

    max_v = sqrt(maximum([v for (_, v) in filtration]))
    min_v = minimum([v for (_, v) in filtration])
    
    f = Figure(fontsize = 12)
    title_fs = 15
    ms = 15
    cm = :Dark2_8
    cr = (1,8)

    i_sc = LScene(f[1:2, 1:2], show_axis=false, scenekw = (lights = [AmbientLight(RGBf(1.0, 1.0, 1.0))],))
    fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
    msh = GeometryBasics.Mesh(bcs, fs)

    clr = [e[2] for e in filtration if length(e[1]) == 1]
    mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)
    if show_wireframe
        wireframe!(i_sc, msh, color=:black)
    end
    f
end

function get_interface_and_multicolored_tetrahedron_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int, mc_tets, show_mc_edges = false, show_wireframe = false, show_barycenters = false)
    bcs, filtration = get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

    get_edges(tet) = [
        (tet[1], tet[2]),
        (tet[1], tet[3]),
        (tet[1], tet[4]),
        (tet[2], tet[3]),
        (tet[2], tet[4]),
        (tet[3], tet[4])
    ]

    f = Figure(fontsize = 12)
    title_fs = 15
    ms = 15
    cm = :Dark2_8
    cr = (1,8)

    i_sc = LScene(f[1:2, 1:2], show_axis=false, scenekw = (lights = [AmbientLight(RGBf(1.0, 1.0, 1.0))],))
    fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
    msh = GeometryBasics.Mesh(bcs, fs)

    clr = [e[2] for e in filtration if length(e[1]) == 1]

    max_v = maximum([c for c in clr])
    min_v = minimum([c for c in clr])

    mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :magma)
    if show_wireframe
        wireframe!(i_sc, msh, color=:white, linewidth = 3)
    end

    if show_barycenters
        for (i, bc) in enumerate(bcs)
            scatter!(i_sc, [bc], color=:black, markersize = ms)
            text!(i_sc, [bc]; text="$(i)", fontsize = 15, color=:red)
        end
    end

    if show_mc_edges
        c0 = RGBf(1.0, 0.0, 0.0)
        c1 = RGBf(0.0, 1.0, 0.0)
        c2 = RGBf(0.0, 0.0, 1.0)
        c3 = RGBf(1.0, 1.0, 0.0)

        for tet in eachrow(mc_tets)
            for (i,j) in get_edges(tet)
                if i > j 
                    i, j = j, i
                end
                if Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([0,1])
                    lines!(i_sc, [points[i], points[j]], color = 1:2, colormap=[c0, c1], linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([0,2])
                    lines!(i_sc, [points[i], points[j]], color = 1:2, colormap=[c0, c2], linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([0,3])
                    lines!(i_sc, [points[i], points[j]], color = 1:2, colormap=[c0, c3], linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([1,2])
                    lines!(i_sc, [points[i], points[j]], color = 1:2, colormap=[c1, c2], linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([1,3])
                    lines!(i_sc, [points[i], points[j]], color = 1:2, colormap=[c1, c3], linewidth=2)            
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([2,3])
                    lines!(i_sc, [points[i], points[j]], color = 1:2, colormap=[c2, c3], linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([0])
                    lines!(i_sc, [points[i], points[j]], color = c0, linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([1])
                    lines!(i_sc, [points[i], points[j]], color = c1, linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([2])
                    lines!(i_sc, [points[i], points[j]], color = c2, linewidth=2)
                elseif Set([div(i-1, n_atoms_per_mol), div(j-1, n_atoms_per_mol)]) == Set([3])
                    lines!(i_sc, [points[i], points[j]], color = c3, linewidth=2)
                end
            end
        end
    end
    f
end

function get_debug_interface_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int)
    mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    bcs, filtration = MorphoMol.Energies.get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

    max_v = maximum([v for (_, v) in filtration])
    min_v = minimum([v for (_, v) in filtration])
    
    f = Figure(fontsize = 12)
    title_fs = 15
    ms = 15
    cm = :Dark2_8
    cr = (1,8)

    i_sc = LScene(f[1:2, 1:2], show_axis=false, scenekw = (lights = [AmbientLight(RGBf(1.0, 1.0, 1.0))],))

    fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
    msh = GeometryBasics.Mesh(bcs, fs)
    clr = [e[2] for e in filtration if length(e[1]) == 1]
    mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)
    for (i, bc) in enumerate(bcs)
        scatter!(i_sc, [bc], color=:black, markersize = ms)
        text!(i_sc, [bc]; text="$(i)", fontsize = 15, color=:red)
    end
    f
end

get_debug_interface_visualization (generic function with 1 method)

In [272]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.0, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 3.0, 1.2]), Point3f([4.0, 2.0, 1.8])]
n_atoms_per_mol = 3
mc_tets = get_multichromatic_tetrahedra(points, n_atoms_per_mol)
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))

GLMakie.Screen(...)

In [273]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.0, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 3.0, 1.2]), Point3f([4.0, 2.0, 1.8]), Point3f(2.5, 2.5, 2.5)]
n_atoms_per_mol = 3
mc_tets = get_multichromatic_tetrahedra(points, n_atoms_per_mol)
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))

GLMakie.Screen(...)

In [ ]:
points = [Point3f([0.1, 0.1, 0.1]), Point3f([2.0, 0.0, 0.2]), Point3f([1.0, 2.5, 0.0]), Point3f([0.0, 0.0, 0.0]), Point3f([1.0, 1.4, 1.0])]
n_atoms_per_mol = 2
mc_tets = Matrix{Int64}([1 2 3 5])
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))

GLMakie.Screen(...)

In [262]:
points = [Point3f([0.1, 0.1, 0.5]), Point3f([2.0, 0.0, 0.2]), Point3f([1.0, 2.5, 0.0]), Point3f([-2.0, 2.0, 0.4]), Point3f([1.0, 1.4, 2.0])]
n_atoms_per_mol = 2
mc_tets = Matrix{Int64}([1 2 3 5; 1 3 4 5])
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))

GLMakie.Screen(...)

In [264]:
points = [Point3f([-2.0, -2.0, 0.4]), Point3f([2.0, -2.0, 0.0]), Point3f([1.4, 0.0, 1.4]), Point3f([-1.4, 0.0, -1.4]), Point3f([1.4, 2.0, -1.4]), Point3f([-1.4, 2.0, 1.4])]
n_atoms_per_mol = 2
mc_tets = get_multichromatic_tetrahedra(points, n_atoms_per_mol)    
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))

GLMakie.Screen(...)

In [266]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.15, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 3.0, 2.4])]
n_atoms_per_mol = 1
mc_tets = get_multichromatic_tetrahedra(points, n_atoms_per_mol)
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))

GLMakie.Screen(...)

In [277]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([0.3, 3.0, -0.5]), Point3f([4.0, 0.0, 0.1]), Point3f([3.5, 2.0, 0.1]), Point3f([0.3, 0.0, 3.5]), Point3f([0.0, 2.0, 3.3]), Point3f([1.1, -2.0, 1.2]), Point3f([-2.0, -2.0, 1.2])]
n_atoms_per_mol = 2
mc_tets = get_multichromatic_tetrahedra(points, n_atoms_per_mol)
display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol, mc_tets, true, true))


GLMakie.Screen(...)

In [276]:
@load "../../Data/hpc_out/scans/2_6r7m/1/11_rwm_ma_2_6r7m.jld2" input output
n_atoms_per_mol = length(input["template_radii"])
points = MorphoMol.Utilities.get_point3f_realization(output["states"][argmin(output["Es"])], input["template_centers"]);


In [275]:
display(get_interface_visualization(points, n_atoms_per_mol, true))

GLMakie.Screen(...)